<a href="https://colab.research.google.com/github/lucasdblucas/deep_learning_skinreader/blob/master/BARBIERI_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# código base -> https://github.com/microsoft/leprosy-skin-lesion-ai-analysis

In [2]:
!pip install pretrainedmodels==0.7.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 7.0 MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=e5180757246d9e93c995a2d6ed9463de17525550415d6b8d0e73e7156bbaec83
  Stored in directory: /root/.cache/pip/wheels/ed/fa/b9/5c82b59d905f95542a192b883c0cc0082407ea2f54beb2f9e6
Successfully built pretrainedmodels


In [3]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.
import copy
import os
import cv2
import pandas as pd
import PIL
from PIL import Image
import torchvision.transforms as transforms
import pretrainedmodels as ptm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
import torch.optim as optim
import numpy as np
from collections import Counter
import random
from random import randint

# específico do prepare data
from torch.utils.data import DataLoader, Dataset
import torchvision
import torch

from datetime import datetime
import json


#Support

In [4]:
metadata_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/dataset/skinreader/BDHansen', 'metadados')
images_closeup_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/dataset/skinreader/BDHansen', 'closeup')
images_all_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/dataset/skinreader/BDHansen', 'all_closeup_panoramic')
images_panoramic_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/dataset/skinreader/BDHansen', 'panoramic')

#Dataframe from .json

In [5]:
def load_jsonfile(json_path):
    with open(json_path,'r') as arquivo:
        paciente = arquivo.read()
        return json.loads(paciente)

In [6]:
def get_json_files():
    files = os.listdir(metadata_path)
    return [load_jsonfile(os.path.join(metadata_path, f)) for f in files if f.endswith('.json')]

In [7]:
def get_data(json_list):
    patient_id_list = []
    image_list_for_patient = []
    for j in json_list:
        patient_id_list.append(j['_via_settings']['project']['name'].split('/')[0])
        list_aux = []
        for key, value in j['_via_img_metadata'].items():
            if len(value['regions']) != 0 :
                # print(value['filename'])
                # print(value['regions'][0])
                list_aux.append((value['filename'], value['regions'][0]['region_attributes']['leprosy']))
        if len(list_aux) != 0: image_list_for_patient.append(list_aux) #lista de imagens e diagnóstico para cada imagem para cada paciente
    
    dados = []
    c_imagenames = os.listdir(images_closeup_path)
    a_imagenames = os.listdir(images_all_path)
    p_imagenames = os.listdir(images_panoramic_path)
    print(f'Numero de imagens -> \nColseup: {len(c_imagenames)}\nAll: {len(a_imagenames)}\nPanoramicas: {len(p_imagenames)}')
    for index, p in enumerate(patient_id_list):
        study_id = p.split('-')[1]

        c_images = [image_name for image_name in c_imagenames if study_id in image_name]
        a_images = [image_name for image_name in a_imagenames if study_id in image_name]
        p_images = [image_name for image_name in p_imagenames if study_id in image_name]

        # for i in c_images:
        #     no_extention = i.split('.')[0]
        #     for n in image_list_for_patient[index]:
        #         if no_extention in n[0]:
        #             dados.append([
        #                 study_id, # ID do estudo
        #                 p, # ID do paciente
        #                 i, # nome da imagem
        #                 n[0], # filename da imagem no json
        #                 os.path.join(images_closeup_path, i), # path da imagem local
        #                 n[1], # dignostico por imagem no json
        #                 'closeup', # tipo de imagem de acordo com a distribuição local
        #                 i.split('.')[0].split('-')[-1] # tipo de imagem de acordo com o estudo
        #             ])
                    
        # for i in p_images: 
        #     no_extention = i.split('.')[0]
        #     for n in image_list_for_patient[index]:
        #         if no_extention in n[0]:
        #             dados.append([study_id, p, i, n[0], os.path.join(images_closeup_path, i), n[1], 'panoramic', i.split('.')[0].split('-')[-1]])

        for i in a_images:
            no_extention = i.split('.')[0]
            for n in image_list_for_patient[index]:
                if no_extention in n[0]:
                    dados.append([study_id, p, no_extention, n[0], os.path.join(images_closeup_path, i), n[1], 'all', i.split('.')[0].split('-')[-1]])

    return dados

#Clean

In [8]:
'''
Clean poly_df
return a dataframe with columns ['patient_id','image_name','patient_leprosy','type']
'''
def clean_poly(dir_name, # os.path.join(dir_name, 'images') contains all the lesion images
               poly_address, # address of labels.tsv
               patient_info_address, # address of patient information csv
               keep_label_id=False # whether to include the column 'label_id' in the output dataframe
              ):
    poly_df = pd.read_csv(poly_address, sep="\t")
    patient_df=pd.read_csv(patient_info_address, engine='python').replace("200AF", "200AP")
    all_images=os.listdir(os.path.join(dir_name, 'images'))
    
    # These three patients dropped the exp. 
    poly_df=poly_df[~poly_df.patient_id.isin(
        ["Patient-051GR", "Patient-121DF","Patient-203PR"])]
    
    # Drop those rows if corresponding images were not found in the image folder
    poly_df=poly_df[poly_df.image_name.isin(all_images)]
    # StudyID é somente a identificação do estudo, ou seja se patiente_id = Patient-051GR, StudyID = 051GR
    StudyID=[str[8:] for str in poly_df['patient_id'].values]

    Diag=[patient_df.loc[patient_df['StudyID']==id,'Diagnostic'].values[0] for id in StudyID]
    Diag_patient=['leprosy' if x<2 else 'other_dermotosis' for x in Diag]
    # Diagnóstico de lepra ou outra dermatite
    poly_df['patient_leprosy']=Diag_patient
    
    #Remove the lesion if diagnostic result of the patient did not match that of the lesion when the lesion diagnostic result is present. 
    rm_label_id=poly_df.loc[(poly_df['patient_leprosy']!=poly_df['lesion_leprosy']) & poly_df['lesion_leprosy'].notna(),"label_id"].values
    if len(rm_label_id)>0: print("Drop these ids because patient_diag is not equal to lesion_diag: "+', '.join(rm_label_id))
    poly_df=poly_df.loc[~ poly_df['label_id'].isin(rm_label_id),]
    poly_df.drop(columns=['lesion_leprosy'],inplace=True)
    
    # Three image types: closeup, panoramic and edge
    types=[x.split('.')[0].split('-')[-1] for x in poly_df.image_name]
    poly_df['type']=types
    poly_df.replace('paoramic','panoramic',inplace=True)
    poly_df.replace('panoramis','panoramic',inplace=True)
    if keep_label_id: return poly_df[['patient_id','image_name','label_id','patient_leprosy','type']].drop_duplicates()
    else:   
        poly_df=poly_df[['patient_id','image_name','patient_leprosy','type']]
        return poly_df.drop_duplicates()




In [9]:
#Reviwed
#return all the image ids of one selected patient
# Documentation pd.loc : https://medium.com/horadecodar/data-science-tips-02-como-usar-loc-e-iloc-no-pandas-fab58e214d87#:~:text=loc%3A,aos%20demais%20usos%2C
def get_img_ids_onepatient(poly_df,patient_id,type_spec=None):
    df=poly_df

    # se "type_spec" for especificado, então df é modificado
    # df é modificado para ser um dataframe somente com o "type" do tipo "type_spec".
    # df.loc -> retornar as linhas onde a coluna type é igual a type_spec, sem especificar colunas, ou seja, retorna todas as colunas
    if type_spec: 
        df=df.loc[df.type==type_spec,]

    # somente os valores da coluna "image_name" para todos os pacientes.
    selected_label_ids=df.loc[df.patient_id==patient_id,].image_name.values
    # selected_label_ids=df.loc[df.patient_id==patient_id,].image_path.values # estou supondo que o nome da imagem seja o seu path.
    return selected_label_ids.tolist()

#return all the image ids of the selected patients
# são todas as imagens. Para cada paciente existem várias imagens.
def get_img_ids(poly_df,patient_ids,type_spec=None):
    selected_label_ids=sum([get_img_ids_onepatient(poly_df,patient,type_spec) for patient in patient_ids],[])
    return selected_label_ids

In [10]:
#Reviwed
'''
split a sequence 'seq' into 'num' sub-sequence
return a list including all the sub-sequences [seq1,...,seq_num] 
'''
# Retorna uma lista (out) de listas. Cada lista dentro de "out" é uma branck com pacientes dentro.
def chunkIt(seq, num):
    avg = int(len(seq) / num)
    out = []
    last = 0.0
    while len(out)<num:
        if len(out)<num-1: out.append(seq[int(last):int(last + avg)])
        else: out.append(seq[int(last):len(seq)])
        last += avg
    return out

#Preprocess

## BARBIERE;2022 Random Transformations

In [ ]:
# Reviwed
# size = igual as dimensões da camada de input
def barbiere2022_random_derive_transform(size, mean, std, scale=.6, change_aspect_ratio=None):
    augs={
        "size":0,
        "mean":0,
        "std":0,
       "color_contrast": 0.3, 
       "color_saturation": 0.3, 
       "color_brightness": 0.3, 
       "color_hue": 0.1, 
       "rotation": 90, 
       "shear": 20
    }
    augs['size'] = size
    augs['mean'] = mean
    augs['std'] = std
    tf_list = []
    # RandomResizedCrop is doing a crop first and then scale to the desired size.
    # [ref: https://pytorch.org/vision/main/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py]
    if not change_aspect_ratio:  tf_list.append(transforms.RandomResizedCrop(augs['size'], scale=(scale, 1), ratio=(1.0, 1.0))) #mantem ratio    
    else: tf_list.append(transforms.RandomResizedCrop(augs['size'], scale=(scale, 1)))
    tf_list.append(transforms.RandomHorizontalFlip())
    tf_list.append(transforms.RandomVerticalFlip())
    tf_list.append(
        transforms.ColorJitter(
            brightness=augs['color_brightness'],
            contrast=augs['color_contrast'],
            saturation=augs['color_saturation'],
            hue=augs['color_hue']
        )
    )
    tf_list.append(transforms.ToTensor())
    tf_augment = transforms.Compose(tf_list)

    train_tf=transforms.Compose([tf_augment,transforms.Normalize(augs['mean'], augs['std'])])
    orig_tf=transforms.Compose([transforms.Resize((augs['size'],augs['size'])),transforms.ToTensor(),transforms.Normalize(augs['mean'], augs['std'])])
    return train_tf,orig_tf

## BARBIERE;2022 Preset Transformations

In [16]:
# # Reviwed
# # size = igual as dimensões da camada de input
# def barbiere2022_preset_derive_transform(size, mean, std, scale=.6, change_aspect_ratio=None):
#     augs={
#         "size":0,
#         "mean":0,
#         "std":0,
#        "color_contrast": 0.3, 
#        "color_saturation": 0.3, 
#        "color_brightness": 0.3, 
#        "color_hue": 0.1, 
#        "rotation": 90, 
#        "shear": 20
#     }
#     augs['size'] = size
#     augs['mean'] = mean
#     augs['std'] = std
#     tf_list = []
#     # RandomResizedCrop is doing a crop first and then scale to the desired size.
#     # [ref: https://pytorch.org/vision/main/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py]
#     if not change_aspect_ratio:  tf_list.append(transforms.RandomResizedCrop(augs['size'], scale=(scale, 1), ratio=(1.0, 1.0))) #mantem ratio    
#     else: tf_list.append(transforms.RandomResizedCrop(augs['size'], scale=(scale, 1)))
#     tf_list.append(transforms.RandomHorizontalFlip())
#     tf_list.append(transforms.RandomVerticalFlip())
#     tf_list.append(
#         transforms.ColorJitter(
#             brightness=augs['color_brightness'],
#             contrast=augs['color_contrast'],
#             saturation=augs['color_saturation'],
#             hue=augs['color_hue']
#         )
#     )
#     tf_list.append(transforms.ToTensor())
#     tf_augment = transforms.Compose(tf_list)

#     train_tf=transforms.Compose([tf_augment,transforms.Normalize(augs['mean'], augs['std'])])
#     orig_tf=transforms.Compose([transforms.Resize((augs['size'],augs['size'])),transforms.ToTensor(),transforms.Normalize(augs['mean'], augs['std'])])
#     return train_tf,orig_tf

## CLARO;2022 Random Transformations

In [11]:
# Reviwed
# size = igual as dimensões da camada de input
def claro2022_random_derive_transform(transformations, size, mean, std, scale=.6, change_aspect_ratio=None):
    augs={
        "size":0,
        "mean":0,
        "std":0,
       "color_contrast": 0.3, 
       "color_saturation": 0.3, 
       "color_brightness": 0.3, 
       "color_hue": 0.1, 
       "rotation": 90, 
       "shear": 20
    }
    augs['size'] = size
    augs['mean'] = mean
    augs['std'] = std
    tf_list = []
    # [ref: https://pytorch.org/vision/main/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py]
    # RandomRotation
    if transformations["rotation"]: tf_list.append(transforms.RandomRotation(degrees=(0, 180)))
    # RandomTranslation
    if transformations["translation"]: tf_list.append(transforms.RandomAffine(degrees=0, translate=(0.35, 0.35)))
    # RandomShear
    if transformations["shear"]: tf_list.append(transforms.RandomAffine(degrees=0, shear=[-50, 50, -50, 50]))
    # RandomFlip
    if transformations["flip"]:
        tf_list.append(transforms.RandomHorizontalFlip())
        tf_list.append(transforms.RandomVerticalFlip())
    # RandomZoom
    if transformations["zoom"]: tf_list.append(transforms.RandomResizedCrop(augs['size'], scale=(scale, 1), ratio=(1.0, 1.0))) #mantem ratio    
    
    tf_list.append(transforms.ToTensor())
    tf_augment = transforms.Compose(tf_list)

    # Resize, to Tensor e Normalização
    train_tf=transforms.Compose([tf_augment,transforms.Normalize(augs['mean'], augs['std'])])
    orig_tf=transforms.Compose([transforms.Resize((augs['size'],augs['size'])),transforms.ToTensor(),transforms.Normalize(augs['mean'], augs['std'])])
    return train_tf, orig_tf

## CLARO;2022 Preset Transformations

In [ ]:
# Reviwed
# size = igual as dimensões da camada de input
def claro2022_preset_derive_transform(transformations, size, mean, std, scale=.6, change_aspect_ratio=None):
    augs={
        "size":0,
        "mean":0,
        "std":0,
       "color_contrast": 0.3, 
       "color_saturation": 0.3, 
       "color_brightness": 0.3, 
       "color_hue": 0.1, 
       "rotation": 90, 
       "shear": 20
    }
    augs['size'] = size
    augs['mean'] = mean
    augs['std'] = std
    tf_list = []
    # [ref: https://pytorch.org/vision/main/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py]
    # RandomRotation
    tf_list.append(transforms.RandomRotation(degrees=(0, 180)))
    # RandomTranslation
    tf_list.append(transforms.RandomAffine(degrees=0, translation=(0.35, 0.35)))
    # RandomShear
    tf_list.append(transforms.RandomAffine(degrees=0, shear=[-50, 50, -50, 50]))
    # RandomFlip
    tf_list.append(transforms.RandomHorizontalFlip())
    tf_list.append(transforms.RandomVerticalFlip())
    # RandomZoom
    tf_list.append(transforms.RandomResizedCrop(augs['size'], scale=(scale, 1), ratio=(1.0, 1.0))) #mantem ratio    
    
    tf_augment = transforms.Compose(tf_list)

    # Resize, to Tensor e Normalização
    train_tf=transforms.Compose([tf_augment,transforms.Normalize(augs['mean'], augs['std'])])
    orig_tf=transforms.Compose([transforms.Resize((augs['size'],augs['size'])),transforms.ToTensor(),transforms.Normalize(augs['mean'], augs['std'])])
    return train_tf, orig_tf

#Prepare Data

In [12]:
class LeprosyDataset(Dataset):
    """Leprosy dataset."""

    def __init__(
        self, 
        img_names, 
        root_dir, 
        img_df,
        target_name, 
        transform=None,
        id_col='label_id',
        extension=".jpg"
    ):
        
        """
        Args:
            img_names (string): List of the names of images in this dataset not including extension
            root_dir (string): Image directory.
            img_df (DataFrame): Dataframe including patient_id, image_name, label_id, lesion_leprosy. Its label_id is identitical to the img_names
            target_name (string): the colname of the response variable in img_df
            transform (callable, optional): Transform to be applied on images.
        Returns:
            image: lesion image
            target: 0(leprosy)/1(nonleprosy)
            weight: prop to 1/(num of images of the patient) 
            freq_ratio:  1/(num of images of the patient) 
        """
        self.img_names = img_names
        self.root_dir = root_dir
        self.img_df=img_df
        self.transform = transform
        self.target_name=target_name
        classes = list(self.img_df[self.target_name].unique())
        classes.sort()
        self.class_to_idx = {classes[i]: i for i in range(len(classes))}
        self.classes = classes
        self.extension=extension
        self.id_col=id_col
        lesion_count=[self.img_df.patient_id.values.tolist().count(p) for p in self.img_df.patient_id] # count num of images for each patient
        count_max=max(lesion_count)# max num of images of one patient
        self.count_max=count_max
        self.img_df['sample_weight']=[count_max/x for x in lesion_count] # give an option to re-weight the loss if there are some patients with siginificantly more images than the other. 
        self.img_df['sample_freq_ratio']=[1/x for x in lesion_count]
        
    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, i):
        image = Image.open(
            os.path.join(
                self.root_dir,
                self.img_names[i]+self.extension
            )
        )
        weight=self.img_df.loc[self.img_df[self.id_col]==self.img_names[i],'sample_weight'].values[0]
        freq_ratio=self.img_df.loc[self.img_df[self.id_col]==self.img_names[i],'sample_freq_ratio'].values[0]
        target = self.class_to_idx[self.img_df.loc[self.img_df[self.id_col]==self.img_names[i],self.target_name].values[0]]
        if self.transform:
            image = self.transform(image)
            
        return image, target,weight,freq_ratio
    
    def numpatient(self):
        return len(set(self.img_df.loc[self.img_df[self.id_col].isin(self.img_names),'patient_id'].values))


#Run Model

In [13]:
def run_model(
    save_dir, # models and result tables will saved in 'save_dir/experiment_name'. 'save_dir' must exists!
    dir_name, # all images are in dir_name/dataset_name
    
    # poly_address, # address of labels.tsv
    # patient_info_address, # address of the patient form csv
    poly_df,

    id=0, # cuda id for parallel experiments
    experiment_name = None, # if None, experiment_name will be the date when exp is finished
    lr=.003, # learning rate
    wd=0, # weight decay
    batch_size=32,
    tuning='tune_all', # default is to train the whole neural net. Otherwise train the last layer only. 
    class_weight="no", # not apply class_weight to the loss. Otherwise do apply. 
    sample_weight="no",# not apply sample_weight to the loss. Otherwise do apply. 
    num_epoch=200, # num of epochs
    model_name='resnet50',# the other option 'inceptionv4'
    transformations=None,
    no_augmentation=False, # if TRUE -> no augmentation at all
    change_aspect_ratio=None, # whether to change aspect ratio
    scale=.6, # scaling factor for data augmentation

    dataset_name='images', # all images are in dir_name/dataset_name
    type_spec=None,# If None, use all three types of images. If ='closeup'/'panoramic'/'edge', only use the specific type of images for model training/validation/testing.
    optim_type="SGD", # otherwise use "Adam" as the optimizer
    num_fold=6, # Split all patients into 6 folds. The last folds will be used to validate Model 3 later. The first 5 folds will be used for 5-fold cross-validation. 
    test_foldid=0 # index of the validation fold. 0-4 only.
):
    if type_spec not in [None,'closeup','panoramic','edge']: raise IOError('Invalid image type!')
    if not os.path.exists(save_dir): raise IOError('save_dir not exisit!')
    device=torch.device("cuda",id)
    # device=torch.cuda.device('cuda:0')

    # poly_df = clean_poly(
    #     dir_name,
    #     poly_address,
    #     patient_info_address
    # )

    # A Index.unique() função Pandas retorna valores únicos no índice. Os únicos são retornados em ordem de aparência, isso NÃO classifica.
    # patient_list - lista com pacientes sem pacientes repetidos.
    patient_list = poly_df.patient_id.unique()
    # patient_list_leprosy = retorna 'patient_id' dos pacientes com diagnóstico = 'leprosy', sem 'pacient_id' repetido. Na sequencia em que aparecem.
    patient_list_leprosy = poly_df.loc[poly_df.patient_leprosy=='leprosy','patient_id'].unique()
    # patiente_list_OD = retorna 'patient_id' dos pacientes com diagnóstico = 'other_dermotosis', sem 'pacient_id' repetido. Na sequencia em que aparecem.
    # OD = Other Dermatosis
    patient_list_OD = poly_df.loc[poly_df.patient_leprosy=='other_dermotosis','patient_id'].unique()
    
    # leprosy_split = patients with leprosy plited in chunks/batchs
    leprosy_split=chunkIt(patient_list_leprosy, num_fold)
    # OD_split = patients with outher dermatosis in chunks/batchs
    OD_split=chunkIt(patient_list_OD, num_fold)

    # Levae the last folder out for validating Model 3
    leprosy_split.pop()
    OD_split.pop()

    # Fold 'test_foldid' for testing, one for model selection by the best accuracy, and the rest for model training. This is a stratified split. 
    # Concatenar representantes das classes (leprosy, other dermatosis)
    test_patient=np.concatenate((leprosy_split.pop(test_foldid), OD_split.pop(test_foldid) ), axis=None)
    val_patient=np.concatenate((leprosy_split.pop(), OD_split.pop()), axis=None)
    train_patient=np.concatenate(leprosy_split+OD_split)
    
    # Drop the three patients without any closeup image
    # não sei se ainda está no dataset, mas será usado em caso ainda constem.
    poly_df=poly_df[~poly_df.patient_id.isin(['Patient-009OS', 'Patient-017SG', 'Patient-076PR'])]
    
    # Get all the image ids of the test/val/train patients
    test_img_ids=get_img_ids(poly_df,test_patient,type_spec)
    train_img_ids=get_img_ids(poly_df,train_patient,type_spec)
    val_img_ids=get_img_ids(poly_df,val_patient,type_spec)
    # poly_df=poly_df.loc[poly_df.image_name.isin(train_img_ids+val_img_ids+test_img_ids)]
    # isin -> verifica se o valor está dentro do dataframe dado. Retorna um dataframe do mesmo tamanho preenchido com Boobleanas.
    # Documentação isin -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html
    # Verificar se as imagens com os id's selecionados estão no dataframe.
    poly_df=poly_df.loc[poly_df.image_name.isin(train_img_ids+val_img_ids+test_img_ids)]

    # train_diag = diagnostico para imagens, onde as imagens estão presentes. Ou seja só retorna o dignótico caso a imagem 
    # exista essa imagem.
    train_diag=poly_df.loc[poly_df.image_name.isin(train_img_ids),'patient_leprosy'].values
    # Conta quantidade de diagnósticos e retorna o máximo entre eles.
    max_cnt=max(train_diag.tolist().count('leprosy'),train_diag.tolist().count('other_dermotosis'))
    # não vou verificar isso agora, porque clss_weight não é utilizado nesse momento.
    # Ao que parece é: o número máximo entre diagnótico de leprosy e outra dermatosi, esse número divido por diagnótico de leprosy e depois dermatosi
    if class_weight!="no": class_weights=torch.FloatTensor([max_cnt/train_diag.tolist().count('leprosy'),
        max_cnt/train_diag.tolist().count('other_dermotosis')]).to(device)


    #escolha do modelo
    # ptm = pretrainedmodels 
    if model_name=='resnet50': 
        model=ptm.resnet50(num_classes=1000, pretrained='imagenet') # uso de uma resnet pretreinada - imagenet
        # model_test = copy.deepcopy(model)
    else: 
        model = ptm.inceptionv4(num_classes=1000, pretrained='imagenet') # se não a resnet50, uso de módulos inception, provavelemnte googlenet - iamgenet
        # model_test = copy.deepcopy(model)
    # github - pretrained models for pytorch -> https://github.com/Cadene/pretrained-models.pytorch#modellast_linear
    # mais sobre pretrained models -> https://discuss.pytorch.org/t/how-to-modify-the-final-fc-layer-based-on-the-torch-model/766/22?page=2
    # last_linear é a ultima camada FC de "model"
    # torck.nn -> These are the basic building blocks for graphs
    # Linear -> Aplica uma transformação linear a entrada
    model.last_linear = nn.Linear(model.last_linear.in_features, 2) #(input, output)
    # Colocar modelo na memória
    if torch.cuda.is_available():model.cuda(id)
    print(f'type model: {type(model)}, \nmodel.input_size[1/: {model.input_size[1]}\n\n')
    # preprocessamento das imagens
    # test_tf = é somente as transformações básicas. Rezise para o tamanhdo do input da rede. Transformação para tensor e normalização ao final.
    # train_tf = são as transformações utilizadas no aumento dos dados.
    do_tranformations = None
    if transformations['type'] == 'r_claro': 
        do_tranformations = claro2022_random_derive_transform
    if transformations['type'] == 'p_claro': 
        do_tranformations = claro2022_preset_derive_transform
    if transformations['type'] == 'r_barbiere': 
        do_tranformations = barbiere2022_random_derive_transform
    if transformations['type'] == 'p_barbiere': 
        # do_tranformations = barbiere2022_preset_derive_transform
        pass
    
    train_tf, test_tf = do_tranformations(
        transformations,
        model.input_size[1], # A imagem sofrerá resize para o tamanho da entrada 
        model.mean, model.std, # valores que serão utilizados para a normalização
        scale, # scala para transformaçõs 
        change_aspect_ratio # muda ou não a taxa de aspecto. Padrão - None
    )
    val_tf = test_tf

    # quando não há aumento de dados, o tipo de transformação que deve ser feita é somente de normalização, a mesma aplicada aos testes.
    if no_augmentation: train_tf = test_tf
    img_address = os.path.join(dir_name, dataset_name)
    # img_address = os.path.join(dir_name, dataset_name)

  
    # Includes only one type of images if specified
    if type_spec: poly_df=poly_df.loc[poly_df.type==type_spec,]
    # Definir dataset de treinamento
    train_data = LeprosyDataset(
        train_img_ids, 
        img_address, 
        poly_df,
        target_name='patient_leprosy',
        transform=train_tf,                 
        id_col='image_name',
        extension=".jpg"
    )
    train_patient_num = train_data.numpatient()
    train_loader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        # num_workers=4
        num_workers=2
    )
    # Definir dataset de validação
    val_data = LeprosyDataset(
        val_img_ids, 
        img_address, 
        poly_df,
        'patient_leprosy',
        val_tf,
        id_col='image_name',
        extension=".jpg"
    )
    val_patient_num = val_data.numpatient()
    val_loader = DataLoader(
        val_data,
        batch_size=batch_size,
        # num_workers=4
        num_workers=2
    )
    # Definir dataset de teste
    test_data = LeprosyDataset(
        test_img_ids, 
        img_address, 
        poly_df,'patient_leprosy',
        test_tf,
        id_col='image_name',
        extension=".jpg"
    )
    test_patient_num = test_data.numpatient()
    test_loader = DataLoader(
        test_data,
        batch_size=batch_size,
        # num_workers=4
        num_workers=2
    )

    # decidir se vai treinar toda a rede ou somente parte dela
    if class_weight=="no":
        criterion = nn.CrossEntropyLoss(reduction='none')
    else: criterion = nn.CrossEntropyLoss(reduction='none', weight=class_weights)
    if(optim_type=="SGD"):
        if tuning=='tune_all':
            optimizer = optim.SGD(
                model.parameters(),
                lr=lr,
                weight_decay=wd
            )
        else: optimizer = optim.SGD(
            model.last_linear.parameters(),
            lr=lr,
            weight_decay=wd
            )
    else:
        if tuning=='tune_all': optimizer = optim.Adam(model.parameters(),lr=lr,weight_decay=wd)
        else: optimizer = optim.Adam(model.last_linear.parameters(),lr=lr,weight_decay=wd)       
    
    
    # definição de variáveis e começo do treinamento
    train_loss, val_loss, test_loss=[],[],[]
    best_val=1000.0
    best_val_acc=0.0
    for epoch in range(num_epoch):  # loop over the dataset multiple times
        running_loss = 0.0
        
        # model training
        model.train()
        for i,data in enumerate(train_loader, 0): # o argumento 0?
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels, weights, ratios = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            weights = weights.float()
            weights = weights.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            if sample_weight != "no": 
                loss = loss * weights
            loss.mean().backward()
            optimizer.step()
            running_loss += loss.sum()
        
        confusion_matrix = torch.zeros(2,2)
        correct, total, correct_p = 0, 0, 0
        running_loss = 0.0
        with torch.no_grad():#Context-manager that disabled gradient calculation. 
                             #Disabling gradient calculation is useful for inference, when you are sure
                             #that you will not call Tensor.backward(). It will reduce memory
                            #consumption for computations that would otherwise have requires_grad=True
            for data in train_loader:
                images, labels, _, ratios = data
                images = images.to(device)
                labels = labels.to(device)
                ratios = ratios.float()
                ratios = ratios.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.sum()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                correct_p += ((predicted == labels).float() * ratios).sum().item() # patient_wise average
                for t, p in zip(labels.view(-1), predicted.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
        print('[%d] loss: %.3f' %(epoch + 1, running_loss))
        print('Accuracy of the network on the training images: %d %%' % (100 * correct / total))
        print(correct_p/train_patient_num)
        train_loss.append([running_loss.item(),100 * correct / total,100*correct_p/train_patient_num]+
                      confusion_matrix.reshape(1,-1)[0].tolist())
        
        # model validation 
        confusion_matrix = torch.zeros(2,2)
        correct,total,correct_p = 0,0,0
        running_loss = 0.0
        model.eval()
        with torch.no_grad():
            for data in val_loader:
                images, labels,_ ,ratios= data
                images=images.to(device)
                labels=labels.to(device)
                ratios=ratios.float()
                ratios=ratios.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.sum()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                correct_p += ((predicted == labels).float()*ratios).sum().item() 
                for t, p in zip(labels.view(-1), predicted.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
        print('Accuracy of the network on the val images: %d %%' % (100 * correct / total))
        print(correct_p/val_patient_num)
        val_loss.append([running_loss.item(), 100 * correct / total, 100*correct_p/val_patient_num]+confusion_matrix.reshape(1,-1)[0].tolist())
        
        # if val_loss<best_val, update best_model
        if running_loss/total<best_val:
            best_model=copy.deepcopy(model).state_dict()
            best_val=running_loss/total
            
        # if val acc>best_val_acc, update best_model_acc
        if correct/total>best_val_acc:
            best_model_acc=copy.deepcopy(model).state_dict()
            best_val_acc=correct/total
        
    # Best_model testing
    confusion_matrix = torch.zeros(2,2)
    correct,total,correct_p = 0,0,0
    running_loss = 0.0
    model.load_state_dict(best_model)
    with torch.no_grad():
        for data in test_loader:
            images, labels, _, ratios = data
            images = images.to(device)
            labels = labels.to(device)
            ratios = ratios.float()
            ratios = ratios.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.sum()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            correct_p += ((predicted == labels).float()*ratios).sum().item() 
            for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
    print('Accuracy of the network on the test images for the best model: %d %%' % (100 * correct / total))
    print(correct_p/test_patient_num)
    test_loss.append([running_loss.item(),100 * correct / total,100*correct_p/test_patient_num]+
                        confusion_matrix.reshape(1,-1)[0].tolist())
    print('Finished Training')
    # Save outputs
    if not experiment_name:
        execution_time = datetime.now().strftime('%d-%m-%Y-%H-%M')
        experiment_name = execution_time
    experiment_folder = os.path.join(save_dir,experiment_name,'fold'+str(test_foldid))
    if not os.path.exists(experiment_folder): os.makedirs(experiment_folder)
    torch.save(model.state_dict(), os.path.join(experiment_folder,'model.pt'))
    torch.save(best_model, os.path.join(experiment_folder,"best_val__model.pt"))
    torch.save(best_model_acc, os.path.join(experiment_folder,"best_val_acc_model.pt"))
    pd.DataFrame(train_loss).to_csv(os.path.join(experiment_folder,'train.csv'),index=False)
    pd.DataFrame(val_loss).to_csv(os.path.join(experiment_folder,'val.csv'),index=False)
    pd.DataFrame(test_loss).to_csv(os.path.join(experiment_folder,'test.csv'),index=False)
    
    """go"""
    print('Acompanhamento do dataset!!')
    # head -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html
    print(poly_df.head(25))
    print('\n\n\n\depois de algumas linhas:')
    print(poly_df.loc[poly_df.image_name == poly_df.image_name[3], "image_path"])

    return None


#Start

## Read JSON files

In [14]:
jsonfiles = get_json_files()
print(f'Patients Num -> {len(jsonfiles)}')
data = get_data(jsonfiles)
print(f'Imagens Num -> {len(data)}')

Patients Num -> 225
Numero de imagens -> 
Colseup: 732
All: 1231
Panoramicas: 499
Imagens Num -> 1205


In [15]:
print(data[0])

['006CJ', 'Patient-006CJ', 'Image-006CJ1-trunk-panoramic', 'https://patientimage.blob.core.windows.net/processedimages/Patient-006CJ/Image-006CJ1-trunk-panoramic.jpg', '/content/drive/MyDrive/Colab Notebooks/dataset/skinreader/BDHansen/closeup/Image-006CJ1-trunk-panoramic.jpg', 'leprosy', 'all', 'panoramic']


In [16]:
df = pd.DataFrame(data, columns=['StudyID', 'patient_id', 'image_name', 'filename', 'image_path', 'patient_leprosy', 'local_dist', 'type'])
df.tail()

,StudyID,patient_id,image_name,filename,image_path,patient_leprosy,local_dist,type
1200,228FM,Patient-228FM,Image-228FM5-llimbs-panoramic,https://patientimage.blob.core.windows.net/pro...,/content/drive/MyDrive/Colab Notebooks/dataset...,leprosy,all,panoramic
1201,225MA,Patient-225MA,Image-225MA2-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,/content/drive/MyDrive/Colab Notebooks/dataset...,other_dermotosis,all,closeup
1202,225MA,Patient-225MA,Image-225MA1-llimbs-panoramic,https://patientimage.blob.core.windows.net/pro...,/content/drive/MyDrive/Colab Notebooks/dataset...,other_dermotosis,all,panoramic
1203,225MA,Patient-225MA,Image-225MA1-llimbs-closeup,https://patientimage.blob.core.windows.net/pro...,/content/drive/MyDrive/Colab Notebooks/dataset...,other_dermotosis,all,closeup
1204,225MA,Patient-225MA,Image-225MA2-trunk-panoramic,https://patientimage.blob.core.windows.net/pro...,/content/drive/MyDrive/Colab Notebooks/dataset...,other_dermotosis,all,panoramic


In [17]:
df.head()

,StudyID,patient_id,image_name,filename,image_path,patient_leprosy,local_dist,type
0,006CJ,Patient-006CJ,Image-006CJ1-trunk-panoramic,https://patientimage.blob.core.windows.net/pro...,/content/drive/MyDrive/Colab Notebooks/dataset...,leprosy,all,panoramic
1,006CJ,Patient-006CJ,Image-006CJ1-trunk-edge,https://patientimage.blob.core.windows.net/pro...,/content/drive/MyDrive/Colab Notebooks/dataset...,leprosy,all,edge
2,006CJ,Patient-006CJ,Image-006CJ1-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,/content/drive/MyDrive/Colab Notebooks/dataset...,leprosy,all,closeup
3,002IR,Patient-002IR,Image-002IR2-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,/content/drive/MyDrive/Colab Notebooks/dataset...,leprosy,all,closeup
4,002IR,Patient-002IR,Image-002IR2-ulimbs-panoramic,https://patientimage.blob.core.windows.net/pro...,/content/drive/MyDrive/Colab Notebooks/dataset...,leprosy,all,panoramic


# Training - Data Augmentation


*   Barbiere
*   All augmentation types



In [14]:
transformations = {
    "type": "r_claro", # "r_barbiere", "p_barbiere", "r_claro", "p_claro"
    "rotations": True,
    "translation": True,
    "shear": True,
    "flip": True,
    "zoom": True,
    "jitter": True
}

In [ ]:
run_model(
    save_dir='/content/drive/MyDrive/Colab Notebooks/skinreadermodels/barbieri2022-info', # models and result tables will saved in 'save_dir/experiment_name'. 'save_dir' must exists!
    dir_name='/content/drive/MyDrive/Colab Notebooks/dataset/skinreader/BDHansen', # all images are in dir_name/dataset_name
    
    poly_df = df,
    id=0, # cuda id for parallel experiments
    experiment_name = None, # if None, experiment_name will be the date when exp is finished
    lr=.003, # learning rate
    wd=0, # weight decay
    batch_size=16,
    tuning='tune_all', # default is to train the whole neural net. Otherwise train the last layer only. 
    class_weight="no", # not apply class_weight to the loss. Otherwise do apply. 
    sample_weight="no",# not apply sample_weight to the loss. Otherwise do apply. 
    num_epoch=10, # num of epochs
    model_name='resnet50',# the other option 'inceptionv4'
    no_augmentation=False, # if TRUE -> no augmentation at all
    transformations=transformations,
    change_aspect_ratio=None, # whether to change aspect ratio
    scale=.6, # scaling factor for data augmentation
    
    dataset_name='all_closeup_panoramic', # all images are in dir_name/dataset_name
    type_spec=None,# If None, use all three types of images. If ='closeup'/'panoramic'/'edge', only use the specific type of images for model training/validation/testing.
    optim_type="SGD", # otherwise use "Adam" as the optimizer
    num_fold=6, # Split all patients into 6 folds. The last folds will be used to validate Model 3 later. The first 5 folds will be used for 5-fold cross-validation. 
    test_foldid=0 # index of the validation fold. 0-4 only.
)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

type model: <class 'torchvision.models.resnet.ResNet'>, 
model.input_size[1/: 224


[1] loss: 436.460
Accuracy of the network on the training images: 47 %
0.4453604854991503
Accuracy of the network on the val images: 50 %
0.44725528959598804
Accuracy of the network on the test images: 50 %
0.4117646988700418
[2] loss: 357.001
Accuracy of the network on the training images: 60 %
0.618398113785503
Accuracy of the network on the val images: 47 %
0.5224852859973907
Accuracy of the network on the test images: 57 %
0.6571895515217501
[3] loss: 351.652
Accuracy of the network on the training images: 62 %
0.592290399787582
Accuracy of the network on the val images: 48 %
0.45439739028612774
Accuracy of the network on the test images: 47 %
0.43251634608296785
[4] loss: 316.573
Accuracy of the network on the training images: 72 %
0.6961731649238929
Accuracy of the network on the val images: 46 %
0.4814803754496906
Accuracy of the network on the test images: 57 %
0.5996965590645286
[5] loss: 294.7

# Training - Data Augmentation


*   Claro
*   All augmentation types
*   Random


In [18]:
transformations = {
    "type": "r_claro", # "r_barbiere", "p_barbiere", "r_claro", "p_claro"
    "rotation": True,
    "translation": True,
    "shear": True,
    "flip": True,
    "zoom": True,
    "jitter": True
}

In [ ]:
run_model(
    save_dir='/content/drive/MyDrive/Colab Notebooks/skinreadermodels/barbieri2022-info', # models and result tables will saved in 'save_dir/experiment_name'. 'save_dir' must exists!
    dir_name='/content/drive/MyDrive/Colab Notebooks/dataset/skinreader/BDHansen', # all images are in dir_name/dataset_name
    
    poly_df = df,
    id=0, # cuda id for parallel experiments
    experiment_name = None, # if None, experiment_name will be the date when exp is finished
    lr=.003, # learning rate
    wd=0, # weight decay
    batch_size=16,
    tuning='tune_all', # default is to train the whole neural net. Otherwise train the last layer only. 
    class_weight="no", # not apply class_weight to the loss. Otherwise do apply. 
    sample_weight="no",# not apply sample_weight to the loss. Otherwise do apply. 
    num_epoch=10, # num of epochs
    model_name='resnet50',# the other option 'inceptionv4'
    no_augmentation=False, # if TRUE -> no augmentation at all
    transformations=transformations,
    change_aspect_ratio=None, # whether to change aspect ratio
    scale=.6, # scaling factor for data augmentation
    
    dataset_name='all_closeup_panoramic', # all images are in dir_name/dataset_name
    type_spec=None,# If None, use all three types of images. If ='closeup'/'panoramic'/'edge', only use the specific type of images for model training/validation/testing.
    optim_type="SGD", # otherwise use "Adam" as the optimizer
    num_fold=6, # Split all patients into 6 folds. The last folds will be used to validate Model 3 later. The first 5 folds will be used for 5-fold cross-validation. 
    test_foldid=0 # index of the validation fold. 0-4 only.
)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

type model: <class 'torchvision.models.resnet.ResNet'>, 
model.input_size[1/: 224


